# KI-Campus - Präprozessierung der Daten

Diese Jupyter Notebook bereitet die Daten für die Lehrveranstaltung vor um das Bearbeiten für die Teilnehmenden zu vereinfachen.


Quelle: https://www.umweltbundesamt.de/daten/luft/luftdaten/luftqualitaet/

In [45]:
import urllib.request
import os
import pandas as pd

In [46]:
data_folder = "data"

try:
    os.mkdir(data_folder)
except FileExistsError:
    pass

## Luftqualitätsdaten beziehen und aufräumen

In [47]:
# Dictionaries mit Stationsname und IDs
stations_and_ids = {
    "Clevischer Ring": "1254",
    "Turiner Strasse" : "1255",
    "Chorweiler": "1108",
    "Rodenkirchen" : "1114"
}

# Datums rage
start_date = "2020-11-01"
end_date = "2021-10-31"

# Für alle Stationen die Daten als CSV-Datei herunterladen
for station, station_id in stations_and_ids.items():
    url = f"https://www.umweltbundesamt.de/api/air_data/v2/airquality/csv?date_from={start_date}&time_from=1&date_to={end_date}&time_to=1&station={station_id}&lang=de"
    urllib.request.urlretrieve(url, f"{data_folder}/{station.replace(' ', '_')}.csv")
    print(f"- Retrieved data for {station}")

- Retrieved data for Clevischer Ring
- Retrieved data for Turiner Strasse
- Retrieved data for Chorweiler
- Retrieved data for Rodenkirchen


In [48]:
# CSV-Datein als DataFrame einlesen
air_qualities_dfs = [
    pd.read_csv(f"{data_folder}/{station.replace(' ', '_')}.csv", 
                sep=";", skipfooter=1, 
                engine="python",  na_values=["-"]) 
    for station in stations_and_ids.keys()
]

air_qualities_dfs[-1].to_csv(f"{data_folder}/Rodenkirchen_processed.csv")

# Alle Datenframe (unterander) zusammenführen.
air_qualities_combined = pd.concat(air_qualities_dfs)

In [49]:
air_qualities_combined

,Stationscode,Datum,Feinstaub (PM₁₀) stündlich gleitendes Tagesmittel in µg/m³,Ozon (O₃) Ein-Stunden-Mittelwert in µg/m³,Stickstoffdioxid (NO₂) Ein-Stunden-Mittelwert in µg/m³,Luftqualitätsindex
0,DENW211,'01.11.2020 01:00',16.0,NaN,18.0,sehr gut
1,DENW211,'01.11.2020 02:00',16.0,NaN,22.0,gut
2,DENW211,'01.11.2020 03:00',16.0,NaN,21.0,gut
3,DENW211,'01.11.2020 04:00',16.0,NaN,22.0,gut
4,DENW211,'01.11.2020 05:00',16.0,NaN,NaN,sehr gut
...,...,...,...,...,...,...
8732,DENW059,'30.10.2021 22:00',11.0,31.0,19.0,sehr gut
8733,DENW059,'30.10.2021 23:00',11.0,29.0,NaN,sehr gut
8734,DENW059,'30.10.2021 24:00',11.0,16.0,21.0,gut
8735,DENW059,'31.10.2021 01:00',10.0,7.0,17.0,sehr gut


In [50]:
# Uhrzeit in eigene Spalte schreiben
air_qualities_combined["Uhrzeit"] = air_qualities_combined["Datum"].apply(
    lambda date: f"{date.split()[1][:-1]}")

# Datumsspalte umformatieren
air_qualities_combined["Datum"] = air_qualities_combined["Datum"].apply(
    lambda date: f"{date.split('.')[2].split()[0]}-{date.split('.')[1]}-{date[1:].split('.')[0]}")

air_qualities_combined = air_qualities_combined[
        ["Stationscode", 
         "Datum", 
         "Uhrzeit",
         "Feinstaub (PM₁₀) stündlich gleitendes Tagesmittel in µg/m³",
         "Ozon (O₃) Ein-Stunden-Mittelwert in µg/m³",
         "Stickstoffdioxid (NO₂) Ein-Stunden-Mittelwert in µg/m³",
         "Luftqualitätsindex"]]

In [51]:
air_qualities_combined

,Stationscode,Datum,Uhrzeit,Feinstaub (PM₁₀) stündlich gleitendes Tagesmittel in µg/m³,Ozon (O₃) Ein-Stunden-Mittelwert in µg/m³,Stickstoffdioxid (NO₂) Ein-Stunden-Mittelwert in µg/m³,Luftqualitätsindex
0,DENW211,2020-11-01,01:00,16.0,NaN,18.0,sehr gut
1,DENW211,2020-11-01,02:00,16.0,NaN,22.0,gut
2,DENW211,2020-11-01,03:00,16.0,NaN,21.0,gut
3,DENW211,2020-11-01,04:00,16.0,NaN,22.0,gut
4,DENW211,2020-11-01,05:00,16.0,NaN,NaN,sehr gut
...,...,...,...,...,...,...,...
8732,DENW059,2021-10-30,22:00,11.0,31.0,19.0,sehr gut
8733,DENW059,2021-10-30,23:00,11.0,29.0,NaN,sehr gut
8734,DENW059,2021-10-30,24:00,11.0,16.0,21.0,gut
8735,DENW059,2021-10-31,01:00,10.0,7.0,17.0,sehr gut


In [52]:
air_qualities_combined.rename(
    columns={"Feinstaub (PM₁₀) stündlich gleitendes Tagesmittel in µg/m³": "Feinstaub",
            "Ozon (O₃) Ein-Stunden-Mittelwert in µg/m³": "Ozon",
            "Stickstoffdioxid (NO₂) Ein-Stunden-Mittelwert in µg/m³": "Stickstoffdioxid"},
    inplace=True)

In [54]:
air_qualities_combined

,Stationscode,Datum,Uhrzeit,Feinstaub,Ozon,Stickstoffdioxid,Luftqualitätsindex
0,DENW211,2020-11-01,01:00,16.0,NaN,18.0,sehr gut
1,DENW211,2020-11-01,02:00,16.0,NaN,22.0,gut
2,DENW211,2020-11-01,03:00,16.0,NaN,21.0,gut
3,DENW211,2020-11-01,04:00,16.0,NaN,22.0,gut
4,DENW211,2020-11-01,05:00,16.0,NaN,NaN,sehr gut
...,...,...,...,...,...,...,...
8732,DENW059,2021-10-30,22:00,11.0,31.0,19.0,sehr gut
8733,DENW059,2021-10-30,23:00,11.0,29.0,NaN,sehr gut
8734,DENW059,2021-10-30,24:00,11.0,16.0,21.0,gut
8735,DENW059,2021-10-31,01:00,10.0,7.0,17.0,sehr gut


In [55]:
air_qualities_combined.dtypes

Stationscode           object
Datum                  object
Uhrzeit                object
Feinstaub             float64
Ozon                  float64
Stickstoffdioxid      float64
Luftqualitätsindex     object
dtype: object

In [58]:
# Kombinierten Daten als CSV-Datei abspeichern
air_qualities_combined.to_csv(f"air_qualities_cologne.csv", index=False)

# Wetterdaten

Leider können die Daten nur direkt heruntergeladen werden (zumindest nicht ohne Tools wie Selenium). 

Daher wurden die Daten von hier manuell als CSV-Datei heruntergeladen:

"https://meteostat.net/de/place/45I0JU?t=2020-11-01/2021-10-31"

In [36]:
weather = pd.read_csv(f"{data_folder}/weather_cologne.csv")

Die Namen der Spalten - hier erläutert:

```
1	date	Datum
2	tavg	Durchschnittstemperatur
3	tmin	Min. Temperatur
4	tmax	Max. Temperatur
5	prcp	Gesamtniederschlag
6	snow	Schneehöhe
7	wdir	Windrichtung
8	wspd	Windgeschwindigkeit
9	wpgt	Spitzenböe
10	pres	Luftdruck
11	tsun	Sonnenscheindauer
```

In [37]:
weather

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2020-11-01,15.5,12.8,18.2,2.3,0,214.4,30.5,63.0,1012.0,0
1,2020-11-02,17.7,11.7,21.2,5.1,0,223.1,37.7,72.0,1010.3,80
2,2020-11-03,10.8,7.3,13.8,0.1,0,219.5,23.5,49.0,1022.6,345
3,2020-11-04,6.7,2.6,11.7,0.0,0,251.0,10.0,32.0,1032.3,428
4,2020-11-05,4.8,0.9,10.3,0.0,0,102.2,6.5,15.0,1038.7,492
...,...,...,...,...,...,...,...,...,...,...,...
360,2021-10-27,13.1,8.9,17.3,0.0,0,182.6,10.2,40.0,1023.6,134
361,2021-10-28,10.8,8.2,15.8,0.0,0,138.8,15.6,47.0,1018.5,532
362,2021-10-29,12.5,8.4,17.5,0.3,0,147.5,18.6,51.0,1010.2,276
363,2021-10-30,12.6,10.8,14.5,2.0,0,152.6,18.4,66.0,1005.4,0


In [38]:
weather.describe()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
count,365.000000,365.000000,365.00000,365.000000,365.000000,364.000000,365.000000,365.000000,365.000000,365.000000
mean,11.495616,7.621918,15.70137,2.510137,1.643836,196.314011,12.488493,39.710411,1016.513699,261.153425
std,6.509197,5.911008,7.56020,9.111566,9.140794,88.982856,6.150639,15.900802,9.259069,253.989959
min,-6.500000,-10.100000,-3.60000,0.000000,0.000000,2.100000,3.400000,13.000000,977.800000,0.000000
25%,6.300000,3.100000,10.10000,0.000000,0.000000,140.700000,8.300000,28.000000,1011.500000,31.000000
50%,11.200000,7.100000,15.60000,0.100000,0.000000,199.700000,11.100000,36.000000,1017.200000,177.000000
75%,17.500000,12.700000,22.00000,2.700000,0.000000,259.300000,15.300000,48.000000,1022.800000,440.000000
max,28.000000,21.600000,34.60000,153.500000,120.000000,358.000000,45.500000,100.000000,1039.700000,930.000000


In [39]:
weather.dtypes

date     object
tavg    float64
tmin    float64
tmax    float64
prcp    float64
snow      int64
wdir    float64
wspd    float64
wpgt    float64
pres    float64
tsun      int64
dtype: object

In [40]:
weather = weather[["date", "tmin", "tmax"]]

In [43]:
weather.to_csv("weather_cologne_selected.csv", index=False)